In [1]:
import os
import numpy as np
from PIL import Image
import sqlite3
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

## データベースへアクセスする

In [5]:
INCLUDED_EXTENTION = [".png", ".jpg"]

# 画像の入っているフォルダを指定し、中身のファイル名を取得
# images.dbを新規作成。images.dbがすでに存在していれば、接続。
dbname = 'images.db'
# データベースへのコネクションオブジェクト作成
conn = sqlite3.connect(dbname)
# sqliteを操作するカーソルオブジェクトを作成
cur = conn.cursor()
# データベースの初期化
cur.execute('DROP TABLE image_info')
# image_infoというtableを作成。
cur.execute('CREATE TABLE image_info (id INTEGER PRIMARY KEY AUTOINCREMENT, filename STRING)')
# データベースへコミットし、変更を保存
conn.commit()
conn.close()

# データベースに画像のファイル名を挿入
conn = sqlite3.connect(dbname)
cur = conn.cursor()
filenames = sorted(os.listdir('handwriting_pics'))
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    cur.execute('INSERT INTO image_info(filename) values(?)', (filename,))
conn.commit()
cur.close()
conn.close()

# tableの中身を取得
conn = sqlite3.connect(dbname)
cur = conn.cursor()
cur.execute('SELECT * FROM image_info')
# fetchall()を使って中身を全て取得
pics_info = cur.fetchall()
cur.close()
conn.close()

## データの前処理をする

In [7]:
img_test = np.empty((0, 64))
#　フォルダ内の全画像をデータ化
for pic_info in pics_info:
    filename = pic_info[1]
    #　画像ファイルを取得、グレースケールにしてサイズ変更
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    img = Image.open(f'handwriting_pics/{filename}').convert('L')
    img_data256 = 255 - np.array(img.resize((8, 8)))

    #画像データ内の最小値が0、最大値が16になるように計算
    min_bright = img_data256.min()
    max_bright = img_data256.max()
    img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
    #加工した画像データの配列をまとめる
    img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]

## 学習と予測

In [9]:
# 教師データ
digits = load_digits()
X = digits.data
y = digits.target
#教師データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
#ロジスティック回帰のモデルの作成し、教師データを使って学習させる
logreg = LogisticRegression(max_iter=2000)
logreg_model = logreg.fit(X_train, y_train)

# 画像データの判別
X_true = []
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    X_true = X_true + [int(filename[:1])]
X_true = np.array(X_true)

#ロジスティック回帰の学習済みモデルに画像データを入れ、判別
pred_logreg = logreg_model.predict(img_test)

print('手書き文字の判別結果')
print('観測結果:', X_true)
print('予測結果:', pred_logreg)
print('正解率:', logreg_model.score(img_test, X_true))


手書き文字の判別結果
観測結果: [0 1 2 3 4 5 6 7 8 9]
予測結果: [4 4 4 4 4 4 4 7 4 4]
正解率: 0.2


/home/umeyuu/flask-study/ml_api/env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 上記のコードをモジュール分割

In [18]:
INCLUDED_EXTENTION = [".png", ".jpg"] # DBにいれる拡張子
dbname = 'images.db' # DBの名前
dir_name = 'handwriting_pics' # 画像があるディレクトリ

# 手書き文字画像が置いてあるパスからファイル名を取得し、リストを作成
def load_filenames(dir_name, included_ext=INCLUDED_EXTENTION):
    files = []
    filenames = sorted(os.listdir(dir_name))
    for filename in filenames:
        base, ext = os.path.splitext(filename)
        if ext not in included_ext:
            continue
        files.append(filename)
    return files

# テーブルを作成する関数
def create_table(dbname):
    conn = sqlite3.connect(dbname)
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS image_info')
    cur.execute( 'CREATE TABLE image_info (id INTEGER PRIMARY KEY AUTOINCREMENT, filename STRING)')
    conn.commit()
    conn.close()
    print("table is successully created")

# 手書き文字画像のファイル名をデータベースに保存
def insert_filenames(dbname, dir_name):
    filenames = load_filenames(dir_name)
    conn = sqlite3.connect(dbname)
    cur = conn.cursor()
    for filename in filenames:
        cur.execute('INSERT INTO image_info(filename) values(?)', (filename,))
    conn.commit()
    cur.close()
    conn.close()
    print("image file names are successully inserted")

# 手書き文字画像のファイル名をデータベースから取得
def extract_filenames(dbname):
    conn = sqlite3.connect(dbname)
    cur = conn.cursor()
    cur.execute( 'SELECT * FROM image_info')
    filenames = cur.fetchall()
    cur.close()
    conn.close()
    return filenames

In [19]:
create_table(dbname)
insert_filenames(dbname, dir_name)
extract_filenames(dbname)

table is successully created
image file names are successully inserted


[(1, '0.jpg'),
 (2, '1.jpg'),
 (3, '2.jpg'),
 (4, '3.jpg'),
 (5, '4.jpg'),
 (6, '5.jpg'),
 (7, '6.jpg'),
 (8, '7.jpg'),
 (9, '8.jpg'),
 (10, '9.jpg')]

In [20]:
# 読み込んだ手書き文字画像の色をグレースケールに変換する関数
def get_grayscale(dir_name):
    filenames = load_filenames(dir_name)
    for filename in filenames:
        img = Image.open(f'{dir_name}/{filename}').convert('L')
        yield img

# 画像サイズを8×8ピクセルのサイズに統一し、明るさも16階調のグレイスケールで白黒に変換する関数
def get_shrinked_img(dir_name):
    img_test = np.empty((0, 64))
    crop_size = 8
    for img in get_grayscale(dir_name):
        img_data256 = 255 - np.array(img.resize((crop_size, crop_size)))
        min_bright, max_bright = img_data256.min(),  img_data256.max()
        img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
        img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]
    return img_test


In [22]:
get_shrinked_img(dir_name)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  8., 16.,  0.,  0.,  0.,  0.,  0.,
         0., 16., 16.,  8.,  0.,  0.,  0.,  0.,  0.,  8.,  8.,  8.,  0.,
         0.,  0.,  0.,  0.,  8., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  3.,
         0.,  0.,  0.,  0.,  0.,  0.,  6.,  3.,  0.,  0.,  0.,  0.,  0.,
         3.,  3.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  3.,  0.,
         0.,  0.,  0.,  3., 16.,  9.,  6.,  0.,  0.,  0.,  

In [23]:

def create_logreg_model():
    digits = load_digits()
    X = digits.data
    y = digits.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    logreg = LogisticRegression(max_iter=2000)
    logreg_model = logreg.fit(X_train, y_train)
    return logreg_model

# テストデータを利用してロジスティック回帰の学習済みモデルのアウトプットを評価
def evaluate_probs(dir_name, img_test, logreg_model):
    filenames = load_filenames(dir_name)
    X_true = [int(filename[:1]) for filename in filenames]  
    X_true = np.array(X_true)
    pred_logreg = logreg_model.predict(img_test)
    
    print('手書き文字の判別結果')
    print('観測結果:', X_true)
    print('予測結果:', pred_logreg)
    print('正解率:', logreg_model.score(img_test, X_true))
    return "Propability calculation is successfully finished"

logreg_model = create_logreg_model()
evaluate_probs(dir_name, img_test, logreg_model)

手書き文字の判別結果
観測結果: [0 1 2 3 4 5 6 7 8 9]
予測結果: [4 4 4 4 4 4 4 7 4 4]
正解率: 0.2


/home/umeyuu/flask-study/ml_api/env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


'Propability calculation is successfully finished'

## 学習済みモデルを作成

In [24]:
import pickle

digits = load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

logreg = LogisticRegression(max_iter=2000)
model = logreg.fit(X_train, y_train)
with open('model.pickle', mode='wb') as fp:
    pickle.dump(model, fp)

/home/umeyuu/flask-study/ml_api/env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
